In [ ]:
#Импорт библиотек

In [1]:
import zipfile


In [13]:
zip_path = "archive (1).zip"  # или полный путь (/content/archive.zip в Colab)
extract_dir = "extracted_folder3"

In [15]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Архив распакован!")

Архив распакован!


In [3]:
pip install nltk

In [4]:
pip install scikeras

Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install keras

In [6]:
pip install tensorflow

In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, LSTM, Dense,Dropout
import numpy as np
from tensorflow.keras.layers import Bidirectional, BatchNormalization, GRU, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
data = pd.read_csv('extracted_folder3/IMDB Dataset.csv')

In [15]:
data=data[0:35000]# возьмем 35000

In [16]:
#Предобработка данных:

In [17]:
def remove1(text):
  p=re.compile("<.*?>")
  return p.sub(r"",text)
data['review']=data['review'].apply(remove1)

In [18]:
def remove_punctuation(text):
  translator=str.maketrans("","",string.punctuation)
  return text.translate(translator)
data["review"]=data["review"].apply(remove_punctuation)

In [19]:
stop_words = set(stopwords.words('english'))
def remove_text(text):
  return " ".join([word for word in text.split() if word.lower() not in stop_words])
data["review"]=data["review"].apply(remove_text)


In [20]:
data.duplicated().sum()

206

In [21]:
data.drop_duplicates(inplace=True)

In [22]:
texts = data['review']
labels =data["sentiment"]

lb = LabelEncoder()
labels = lb.fit_transform(labels)


# Токенизация текста
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review'])
x_seq = tokenizer.texts_to_sequences(texts)


max_length = max(len(seq) for seq in x_seq)
x_pad = pad_sequences(x_seq, maxlen=max_length, padding='post')

In [23]:
#Разделение на траин, тест,вал

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    x_pad, labels, test_size=0.3, random_state=42)

In [25]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [26]:
X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test_cat, test_size=0.5, random_state=42)

In [ ]:
#Создание модели и ее обучение

In [60]:
from tensorflow.keras.layers import GRU, LayerNormalization
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 96
max_length = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
model.add(SpatialDropout1D(0.39))


model.add(Bidirectional(GRU(128, return_sequences=True, recurrent_dropout=0.3, dropout=0.37)))
model.add(LayerNormalization())
model.add(Dropout(0.55))

model.add(Bidirectional(GRU(64)))
model.add(LayerNormalization())
model.add(Dropout(0.55))

model.add(Dense(64, activation='swish'))
model.add(Dropout(0.4))
model.add(Dense(y_train_cat.shape[1], activation='softmax'))

# Оптимизатор
optimizer = Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=2),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5)
]

history = model.fit(
    X_train, y_train_cat,
    batch_size=128,
    epochs=3,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

D:\anaconda 2\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 16282s 85s/step - accuracy: 0.5135 - loss: 0.8231 - val_accuracy: 0.5005 - val_loss: 0.6936 - learning_rate: 0.0010
Epoch 2/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 16245s 85s/step - accuracy: 0.5759 - loss: 0.6647 - val_accuracy: 0.8354 - val_loss: 0.3903 - learning_rate: 0.0010
Epoch 3/3
191/191 ━━━━━━━━━━━━━━━━━━━━ 16818s 88s/step - accuracy: 0.8854 - loss: 0.3019 - val_accuracy: 0.8810 - val_loss: 0.2880 - learning_rate: 0.0010


In [ ]:
#сохранение модели(Случайно нажал на код после удаления сре

In [1]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Тестовая потеря: {test_loss}, Тестовая точность: {test_accuracy}")

# Сохранение модели
model.save('my_model4.h5')

NameError: name 'model' is not defined

In [36]:
import keras
model = keras.models.load_model('my_model4.h5')
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Тестовая потеря: {test_loss}, Тестовая точность: {test_accuracy}")

164/164 ━━━━━━━━━━━━━━━━━━━━ 78s 443ms/step - accuracy: 0.8941 - loss: 0.2754
Тестовая потеря: 0.2632194757461548, Тестовая точность: 0.8973180055618286
